# Raster aggregation

This notebook contains code for aggregating (resampling down) rasters, for example to convert the MODIS 30 arcsecond (~1km) grids into 2.5 arcminute (~5km) grids.

Code is provided for aggregating continuous or categorical rasters, along with various example input commands.

Continuous rasters can be aggregated to produce any/all of count, max, mean, min, range, sum, or std dev of the input cells.

Categorical rasters can be aggregated to produce one class proportion and one like adjacency grid for each of the input values, and a single majority grid. This assumes that the input grids have a small-ish number of unique values, and that they start from zero (it was written for the MCD12Q1 BRDF landcover data).

In [3]:
from osgeo import gdal
import numpy as np
#import scipy.ndimage as ndi
%load_ext cython
import glob
import os
import rasterio

# Aggregation functions

In [2]:

%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
import numpy as np
cimport openmp
from cython.parallel import parallel, prange
from libc.math cimport sqrt
   
@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
cpdef aggregateContinuous(float [:,::1] data, int fact, float _NDV = np.inf, char minMaxRangeSumOnly = 0):
    ''' Aggregates a continuous raster grid by a specified factor (e.g. 10 to aggegate 500m to 5km data)

     Returns a tuple containing up to seven grids at the aggregated resolution, 
     each representing a different summary of the source pixels covered by each output
     pixel, namely:
        (
          Count, 
          Max, 
          Mean (or None), 
          Min, 
          Range, 
          Sum, 
          SD (or None)
        )
     The input grid must (in this implementation) have dimensions that are 
     exact multiples of aggregation factor.
    '''    
    cdef:
        Py_ssize_t xShapeIn, yShapeIn, xShapeOut, yShapeOut
        Py_ssize_t xIn, yIn, xOut, yOut, catNum
        int yBelow, yAbove, xLeft, xRight
        float localValue
        float[:,::1] outputMeanArr
        float[:,::1] outputMinArr
        float[:,::1] outputMaxArr
        float[:,::1] outputRangeArr
        float[:,::1] outputSumArr
        float[:,::1] outputSDArr
        
        float[:,::1] _oldSDArr
        float[:,::1] _oldMeanArr
        
        int[:,::1] outputCountArr
        float proportion
        double variance
        
    yShapeIn = data.shape[0]
    xShapeIn = data.shape[1]
    
    assert fact > 0
    assert yShapeIn % fact == 0
    assert xShapeIn % fact == 0
    
    # how much of an output cell does each input cell account for
    proportion = 1.0 / (fact**2)
    
    yShapeOut = yShapeIn / fact
    xShapeOut = xShapeIn / fact
    
    outputMinArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    outputMaxArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    
    if not minMaxRangeSumOnly:
        outputMeanArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
        outputSDArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    
        _oldSDArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
        _oldMeanArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
        
        outputMeanArr[:] = _NDV
        outputSDArr[:] = _NDV
    
        _oldMeanArr[:] = _NDV
        _oldSDArr[:] = _NDV
    
    
    outputMinArr[:] = np.inf
    outputMaxArr[:] = -np.inf
    
    outputSumArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    outputRangeArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    outputCountArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.int32)
    
    #with nogil, parallel():
    # no parallel written this way round as the min/max values would be non deterministic
    if 1:
        #yOut = -1
        #xOut = -1
        #localValue = -1
        for yIn in range(yShapeIn):
            yAbove = yIn - 1
            if yIn == 0:
                yAbove = -1
            yBelow = yIn+1
            if yIn == yShapeIn-1:
                yBelow = -1
            yOut = <int> yIn / fact
            localValue=-1
            xOut = -1
            for xIn in range(xShapeIn):
                xLeft = xIn - 1
                if xIn == 0:
                    xLeft = -1
                xRight = xIn+1
                if xIn == xShapeIn-1:
                    xRight = -1
                xOut = <int> xIn / fact
                
                localValue = data[yIn, xIn]
                if localValue == _NDV:
                    continue
                # Max and Min
                if localValue > outputMaxArr[yOut, xOut]:
                    outputMaxArr[yOut, xOut] = localValue
                if localValue < outputMinArr[yOut, xOut]:
                    outputMinArr[yOut, xOut] = localValue
                # Sum and Count
                outputSumArr[yOut, xOut] += localValue
                outputCountArr[yOut, xOut] += 1
                if not minMaxRangeSumOnly:
                    # Running mean and SD
                    if outputCountArr[yOut, xOut] == 1:
                        _oldMeanArr[yOut, xOut] = localValue
                        outputMeanArr[yOut, xOut] = localValue
                        _oldSDArr[yOut, xOut] = 0
                        outputSDArr[yOut, xOut] = 0
                    else:
                        outputMeanArr[yOut, xOut] = (_oldMeanArr[yOut, xOut] + 
                                                     ((localValue - _oldMeanArr[yOut, xOut]) / 
                                                          outputCountArr[yOut, xOut]))
                        outputSDArr[yOut, xOut] = (_oldSDArr[yOut, xOut] +
                                                   ((localValue - _oldMeanArr[yOut, xOut]) *
                                                    (localValue - outputMeanArr[yOut, xOut])
                                                    ))
                        _oldMeanArr[yOut, xOut] = outputMeanArr[yOut, xOut]
                        _oldSDArr[yOut, xOut] = outputSDArr[yOut, xOut]

    for yOut in range(yShapeOut): # not bothering to parallelise this, there are frac**2 fewer cells than before
        xOut = -1
        for xOut in range(xShapeOut):
            if outputCountArr[yOut, xOut] == 0:
                outputMinArr[yOut, xOut] = _NDV
                outputMaxArr[yOut, xOut] = _NDV
                outputRangeArr[yOut, xOut] = _NDV
                outputSumArr[yOut, xOut] = _NDV
                if not minMaxRangeSumOnly:
                    outputMeanArr[yOut, xOut] = _NDV
                #continue
            else:
                # min, max, sum, count are already set
                outputRangeArr[yOut, xOut] = outputMaxArr[yOut, xOut] - outputMinArr[yOut, xOut]
                if not minMaxRangeSumOnly:
                    variance = outputSDArr[yOut, xOut] / outputCountArr[yOut, xOut]
                    outputSDArr[yOut, xOut] = sqrt(variance)
                    # re-calculate the mean using simple sum/n as the running mean method is more 
                    # likely to have accumulated (slight) fp errors (in practice they seem to match 
                    # to around 1e-6 but this will depend on the size of the values)
                    outputMeanArr[yOut, xOut] = outputSumArr[yOut, xOut] / outputCountArr[yOut, xOut]
    #return np.round(np.asarray(outputArr)).astype(np.uint8)
    if not minMaxRangeSumOnly:
        return { # count, max, mean, min, range, sum
            "count": np.asarray(outputCountArr),
            "max": np.asarray(outputMaxArr),
            "mean": np.asarray(outputMeanArr).astype(np.float32),
            "min": np.asarray(outputMinArr),
            "range": np.asarray(outputRangeArr),
            "sum": np.asarray(outputSumArr),
            "sd": np.asarray(outputSDArr).astype(np.float32),
        }
    else:
        return {
            "count": np.asarray(outputCountArr),
            "max": np.asarray(outputMaxArr),
            "mean": None, #np.asarray(outputMeanArr).astype(np.float32),
            "min": np.asarray(outputMinArr),
            "range": np.asarray(outputRangeArr),
            "sum": np.asarray(outputSumArr),
            "sd": None # np.asarray(outputSDArr).astype(np.float32),
        }

In [ ]:

%%cython --compile-args=/openmp --link-args=/openmp --force --annotate
cimport cython
import numpy as np
cimport openmp
from cython.parallel import parallel, prange
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
cdef class RasterAggregator_float:
    ''' Aggregates a continuous raster grid by a specified factor (e.g. 10 to aggegate 500m to 5km data)

     Returns a tuple containing up to seven grids at the aggregated resolution, 
     each representing a different summary of the source pixels covered by each output
     pixel, namely:
        (
          Count, 
          Max, 
          Mean (or None), 
          Min, 
          Range, 
          Sum, 
          SD (or None)
        )
    '''    
   
    cdef:
        Py_ssize_t xShapeOut, yShapeOut
        Py_ssize_t xShapeIn, yShapeIn, tileXShapeIn, tileYShapeIn
        
    cdef:
        
        float _NDV
        char minMaxRangeSumOnly
        
        float[:,::1] outputMeanArr
        float[:,::1] outputMinArr
        float[:,::1] outputMaxArr
        float[:,::1] outputRangeArr
        float[:,::1] outputSumArr
        float[:,::1] outputSDArr
        
        float[:,::1] _oldSDArr
        float[:,::1] _oldMeanArr
        
        int[:,::1] outputCountArr
        
        char[:,::1] _coverageArr
        
        float proportion
        double variance
        double xFact, yFact
    
    @cython.boundscheck(False)
    @cython.cdivision(True)
    @cython.wraparound(False)
    def __cinit__(self, xSizeIn, ySizeIn, xSizeOut, ySizeOut, _NDV, minMaxRangeSumOnly):
        assert xSizeIn > xSizeOut
        assert ySizeIn > ySizeOut
        
        self.xShapeIn = xSizeIn
        self.yShapeIn = ySizeIn
        self.xShapeOut = xSizeOut
        self.yShapeOut = ySizeOut
        
        self.minMaxRangeSumOnly = minMaxRangeSumOnly
        
        self.xFact = <double>self.xShapeIn / self.xShapeOut
        self.yFact = <double>self.yShapeIn / self.yShapeOut
        
        # how much of an output cell does each input cell account for
        self.proportion = 1.0 / (self.xFact * self.yFact)
        
        # initialise the output arrays
        self.outputMinArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
        self.outputMaxArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
        self.outputSumArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
        self.outputRangeArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
        self.outputCountArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.int32)
        self.outputMinArr[:] = np.inf
        self.outputMaxArr[:] = -np.inf
        self._coverageArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.byte)
        
        if not minMaxRangeSumOnly:
            self.outputMeanArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
            self.outputSDArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
    
            self._oldSDArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
            self._oldMeanArr = np.zeros(shape=(ySizeOut, xSizeOut), dtype = np.float32)
        
            self.outputMeanArr[:] = _NDV
            self.outputSDArr[:] = _NDV
    
            self._oldMeanArr[:] = _NDV
            self._oldSDArr[:] = _NDV
    
    @cython.boundscheck(False)
    @cython.cdivision(True)
    @cython.wraparound(False)
    cpdef addTile(self, float[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
        cdef:
            Py_ssize_t tileYShapeIn, tileXShapeIn
            int yBelow, yAbove, xLeft, xRight
            Py_ssize_t xInGlobal, xInTile, yInGlobal, yInTile
            float localValue
            Py_ssize_t xOut, yOut
        tileYShapeIn = data.shape[0]
        tileXShapeIn = data.shape[1]
        
        for yInTile in range(tileYShapeIn):
            yInGlobal = yInTile + yOffset
            yAbove = yInGlobal - 1
            if yInGlobal == 0:
                yAbove = -1
            yBelow = yInGlobal+1
            if yInGlobal == self.yShapeIn-1:
                yBelow = -1
            yOut = <int> (yInGlobal / self.yFact)
            localValue=-1
            xOut = -1
            for xInTile in range(tileXShapeIn):
                xInGlobal = xInTile + xOffset
                xLeft = xInGlobal - 1
                if xInGlobal == 0:
                    xLeft = -1
                xRight = xInGlobal + 1
                if xInGlobal == self.xShapeIn - 1:
                    xRight = -1
                xOut = <int> (xInGlobal / self.xFact)
                
                self._coverageArr[yOut, xOut] = 1
                
                localValue = data[yInTile, xInTile]
                if localValue == self._NDV:
                    continue
                # Max and Min
                if localValue > self.outputMaxArr[yOut, xOut]:
                    self.outputMaxArr[yOut, xOut] = localValue
                if localValue < self.outputMinArr[yOut, xOut]:
                    self.outputMinArr[yOut, xOut] = localValue
                # Sum and Count
                self.outputSumArr[yOut, xOut] += localValue
                self.outputCountArr[yOut, xOut] += 1
                if not self.minMaxRangeSumOnly:
                    # Running mean and SD
                    if self.outputCountArr[yOut, xOut] == 1:
                        self._oldMeanArr[yOut, xOut] = localValue
                        self.outputMeanArr[yOut, xOut] = localValue
                        self._oldSDArr[yOut, xOut] = 0
                        self.outputSDArr[yOut, xOut] = 0
                    else:
                        self.outputMeanArr[yOut, xOut] = (self._oldMeanArr[yOut, xOut] + 
                                                     ((localValue - self._oldMeanArr[yOut, xOut]) / 
                                                          self.outputCountArr[yOut, xOut]))
                        self.outputSDArr[yOut, xOut] = (self._oldSDArr[yOut, xOut] +
                                                   ((localValue - self._oldMeanArr[yOut, xOut]) *
                                                    (localValue - self.outputMeanArr[yOut, xOut])
                                                    ))
                        self._oldMeanArr[yOut, xOut] = self.outputMeanArr[yOut, xOut]
                        self._oldSDArr[yOut, xOut] = self.outputSDArr[yOut, xOut]
    
    @cython.boundscheck(False)
    @cython.cdivision(True)
    @cython.wraparound(False)
    cdef finalise(self):
        cdef:
            Py_ssize_t xOut, yOut
            float variance
            float iscomplete = 1
        for yOut in range(self.yShapeOut): # not bothering to parallelise this, there are frac**2 fewer cells than before
            xOut = -1
            if self._coverageArr[yOut, xOut] == 0:
                iscomplete = 0
            for xOut in range(self.xShapeOut):
                if self.outputCountArr[yOut, xOut] == 0:
                    self.outputMinArr[yOut, xOut] = self._NDV
                    self.outputMaxArr[yOut, xOut] = self._NDV
                    self.outputRangeArr[yOut, xOut] = self._NDV
                    self.outputSumArr[yOut, xOut] = self._NDV
                    if not self.minMaxRangeSumOnly:
                        self.outputMeanArr[yOut, xOut] = self._NDV
                    #continue
                else:
                    # min, max, sum, count are already set
                    self.outputRangeArr[yOut, xOut] = (
                        self.outputMaxArr[yOut, xOut] - self.outputMinArr[yOut, xOut])
                    if not self.minMaxRangeSumOnly:
                        variance = self.outputSDArr[yOut, xOut] / self.outputCountArr[yOut, xOut]
                        self.outputSDArr[yOut, xOut] = sqrt(variance)
                        # re-calculate the mean using simple sum/n as the running mean method is more 
                        # likely to have accumulated (slight) fp errors (in practice they seem to match 
                        # to around 1e-6 but this will depend on the size of the values)
                        self.outputMeanArr[yOut, xOut] = (
                            self.outputSumArr[yOut, xOut] / self.outputCountArr[yOut, xOut])
        if not iscomplete:
            print "Warning, generating a result without having received input data for full extent"
    
    @cython.boundscheck(False)
    @cython.cdivision(True)
    @cython.wraparound(False)
    cpdef GetResults(self):
        self.finalise()
        #return np.round(np.asarray(outputArr)).astype(np.uint8)
        if not self.minMaxRangeSumOnly:
            return { # count, max, mean, min, range, sum
                "count": np.asarray(self.outputCountArr),
                "max": np.asarray(self.outputMaxArr),
                "mean": np.asarray(self.outputMeanArr).astype(np.float32),
                "min": np.asarray(self.outputMinArr),
                "range": np.asarray(self.outputRangeArr),
                "sum": np.asarray(self.outputSumArr),
                "sd": np.asarray(self.outputSDArr).astype(np.float32),
            }
        else:
            return {
                "count": np.asarray(self.outputCountArr),
                "max": np.asarray(self.outputMaxArr),
                "min": np.asarray(self.outputMinArr),
                "range": np.asarray(self.outputRangeArr),
                "sum": np.asarray(self.outputSumArr),
            }

                           

In [3]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
import numpy as np
cimport openmp
from cython.parallel import parallel, prange

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
# aggregates a grid by a specified factor (e.g. 10 to aggegate 500m to 5km data),
# producing for each class specified in the input data a grid at the output 
# resolution for each of fraction and like adjacency.
# Like adjacencies are calculated using double-count method whereas Dan's IDL
# code used single-count method, so results are slightly different.
# Designed for use with MCD12Q1 IBGP landcover data, a couple of classes are hardcoded
# to suit this, but they could easily be modified.
cpdef aggregateStats(unsigned int[:,::1] landCover, int fact, int nCategories, float fltNDV = -9999):
    ''' Aggregates a categorical raster grid by a specified factor (e.g. 10 to aggegate 500m to 5km data)

     Returns a tuple containing two three dimensional grids and one two dimensional grid 
     at the aggregated resolution, each representing a different summary of the source 
     pixels covered by each output pixel, namely:
        (
          Fraction (3D), 
          Like-Adjacency (3D), 
          Majority (2D)
        )
     The fraction and like adjacency outputs have one image (in the z dimension) for each category
     (value) of the input raster. It's assumed that the input has values from zero and the mapping 
     is therefore between the input value and the output position, i.e. an input pixel of value 4
     will contribute to the 4th grid in the output stack. This is suitable for use with IGBP 
     landcover data.

     The input grid must (in this implementation) have dimensions that are 
     exact multiples of aggregation factor.
    '''    
    cdef:
        Py_ssize_t xShapeIn, yShapeIn, xShapeOut, yShapeOut
        Py_ssize_t xIn, yIn, xOut, yOut, catNum
        int yBelow, yAbove, xLeft, xRight
        unsigned char localValue
        float[:,:,::1] outputFracArr
        float[:,:,::1] outputLikeAdjArr
        unsigned int [:,::1] outputMajorityArr
        float [:,::1] tmpMajorityPropArr
        float proportion
        
    yShapeIn = landCover.shape[0]
    xShapeIn = landCover.shape[1]
    
    assert fact > 0
    assert yShapeIn % fact == 0
    assert xShapeIn % fact == 0
    
    # how much of an output cell does each input cell account for
    proportion = 1.0 / (fact**2)
    
    yShapeOut = yShapeIn / fact
    xShapeOut = xShapeIn / fact
    outputFracArr = np.zeros(shape=(nCategories, yShapeOut, xShapeOut), dtype = np.float32)
    outputLikeAdjArr = np.zeros(shape=(nCategories, yShapeOut, xShapeOut), dtype = np.float32)
    outputMajorityArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.int32)
    tmpMajorityPropArr = np.zeros(shape=(yShapeOut, xShapeOut), dtype = np.float32)
    
    with nogil, parallel():
        #yOut = -1
        #xOut = -1
        #localValue = -1
        for yIn in prange(yShapeIn):
            yAbove = yIn - 1
            if yIn == 0:
                yAbove = -1
            yBelow = yIn+1
            if yIn == yShapeIn-1:
                yBelow = -1
            yOut = <int> yIn / fact
            localValue=-1
            xOut = -1
            for xIn in range(xShapeIn):
                xLeft = xIn - 1
                if xIn == 0:
                    xLeft = -1
                xRight = xIn+1
                if xIn == xShapeIn-1:
                    xRight = -1
                xOut = <int> xIn / fact
                
                # fractional landcover
                localValue = landCover[yIn, xIn]
                # hardcode a remap from the high values into the next available smallest ones
                if localValue == 254:
                    localValue = 17
                elif localValue == 255:
                    localValue = 18
                outputFracArr [localValue, yOut,xOut] += proportion
                
                # like-adjacencies
                if yAbove>=0 and landCover[yAbove,xIn] == localValue:
                    outputLikeAdjArr[localValue,yOut,xOut] += 0.25 * proportion
                if xRight>=0 and landCover[yIn,xRight] == localValue:
                    outputLikeAdjArr[localValue,yOut,xOut] += 0.25 * proportion
                if yBelow>=0 and landCover[yBelow,xIn] == localValue:
                    outputLikeAdjArr[localValue,yOut,xOut] += 0.25 * proportion
                if xLeft>=0 and landCover[yIn,xLeft] == localValue:
                    outputLikeAdjArr[localValue,yOut,xOut] += 0.25 * proportion

    for catNum in range (nCategories):
        yOut = -1
        xOut = -1
        for yOut in range(yShapeOut): # not bothering to parallelise this, there are frac**2 fewer cells than before
            for xOut in range(xShapeOut):
                # like adjacencies are output as a fraction of the cells that were of a class, that
                # have neighbours of the same class
                if outputFracArr[catNum, yOut, xOut] > 0:
                    outputLikeAdjArr[catNum, yOut, xOut] = (
                        outputLikeAdjArr[catNum, yOut, xOut] / outputFracArr[catNum, yOut, xOut])
                else:
                    outputLikeAdjArr[catNum, yOut, xOut] = fltNDV
                    
                if outputFracArr[catNum, yOut, xOut] > tmpMajorityPropArr[yOut, xOut]:
                    tmpMajorityPropArr[yOut, xOut] = outputFracArr[catNum, yOut, xOut]
                    if catNum < 17:
                        outputMajorityArr[yOut, xOut] = catNum
                    elif catNum == 17:
                        outputMajorityArr[yOut, xOut] = 254
                    elif catNum == 18:
                        outputMajorityArr[yOut, xOut] = 255

                # output in percent (so we can use int array = smaller)
                # it currently contains proportion of the output cell that is covered 
                # by input cells of that type, i.e. a value 0-1
                # and since we are running with frac=10 so 100 inputs to each output, 
                # there can only be a whole percentage value
                outputFracArr[catNum,yOut,xOut] *=100
                # C round function won't import into cython on my machine for some reason so
                # add 0.5 so that truncating downwards has the same effect
                outputFracArr[catNum,yOut,xOut]+=0.5
                
    #return np.round(np.asarray(outputArr)).astype(np.uint8)
    return (
        np.asarray(outputFracArr).astype(np.uint8),
        np.asarray(outputLikeAdjArr),
        np.asarray(outputMajorityArr).astype(np.uint8)
    )
    

In [36]:
def savecontResult(thing,stat, tag, data,_NDV, outDir, outName = None, decrepit=False):
    '''
    Save a result from the continuous raster aggregation function, using the MAP filename formats
    '''
    if data.dtype==np.float32 or data.dtype==np.float64:
        gdType = gdal.GDT_Float32
    elif data.dtype == np.int8 or data.dtype == np.uint8 or data.dtype == np.byte:
        gdType = gdal.GDT_Byte
    else:
        gdType = gdal.GDT_Int32
    
    outDrv = gdal.GetDriverByName('GTiff')
    if outName is None:
        outNameTemplate = r'{0!s}\{1!s}_5km_{2!s}{3!s}.tif'
        outRasterName = outNameTemplate.format(outDir, thing, stat, tag)
    else:
        outRasterName = os.path.join(outDir, outName)
        outRasterName = outRasterName + '.tif'
    cOpts = ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"]
    if decrepit:
        cOpts = []
    outRaster = outDrv.Create(outRasterName,data.shape[1], data.shape[0],1,gdType,
                              cOpts)
    outRaster.SetGeoTransform(outputGT)
    outRaster.SetProjection(inputProj)
    outBand = outRaster.GetRasterBand(1)
    if _NDV:
        outBand.SetNoDataValue(_NDV)
    outBand.WriteArray(data)
    outBand.FlushCache()
    del outBand
    outRaster = None

resStructure = ('Count','Max','Mean','Min','Range','Sum','SD')


# To enforce alignment with the mastergrids run the following on each file:
# gdal_edit %filename.tif% -a_ullr -180 89.99994 179.9998560 -89.999988

# Code to run aggregation functions

### Define aggregation factor

In [5]:
aggFactor = 5


In [6]:
outDir = r'C:\temp'

## Example: run for one continuous dataset

In [32]:
dataPath = r'C:\Users\zool1301.NDPH\Documents\Other_Data\GUF\GUF28\GUF28\000777777786422\0007files_aligned.tif'
ds = gdal.Open(dataPath)
bnd = ds.GetRasterBand(1)
_NDV = bnd.GetNoDataValue()
inputGT = ds.GetGeoTransform()
inputProj = ds.GetProjection()
inputHeightPx = ds.RasterYSize
inputWidthPx = ds.RasterXSize

inputXMin = inputGT[0]
inputYMax = inputGT[3]
inputXMax = inputXMin + inputGT[1] * inputWidthPx
inputYMin = inputYMax + inputGT[5] * inputHeightPx
inputHeightProj = inputYMax - inputYMin
inputWidthProj = inputXMax - inputXMin

desiredOutputRes = 0.008333333333333
desiredOutputWidth = int(inputWidthProj / desiredOutputRes)
desiredOutputHeight = int(inputHeightProj / desiredOutputRes)

outputGT = (inputXMin, desiredOutputRes, 0.0, inputYMax, 0.0, -desiredOutputRes)

In [ ]:
dataPath = r'C:\Users\zool1301\Documents\Other_Data\Ferranti_Elev_15Sec\ferranti15sec_hillshade_Illum360_scale.tif'


In [ ]:
ds = gdal.Open(dataPath)
bnd = ds.GetRasterBand(1)
_NDV = bnd.GetNoDataValue()
inputGT = ds.GetGeoTransform()
inputProj = ds.GetProjection()
outputGT = (inputGT[0], inputGT[1]*aggFactor, 0.0, inputGT[3], 0.0, inputGT[5] * aggFactor)

In [13]:
outputGT

(-180.0, 0.041666666666665, 0.0, 90.0, 0.0, -0.041666666666665)

In [ ]:
#xLims = (21840*2,33040*2) # for 1km data
#yLims = (9660*2,20560*2) # for 1km data
#testArea = bnd.ReadAsArray(xLims[0],yLims[0],xLims[1]-xLims[0],yLims[1]-yLims[0])
allData = bnd.ReadAsArray()

In [ ]:
allData[np.isnan(allData)] = _NDV

#### Data shape must be a clean muliple of aggregation factor, here we just fudge by cutting the end off

In [ ]:
allData = allData[0:-1, 0:-1]

In [ ]:
outputGT2 = (-180.0, 0.008333333333333, 0.0, 90.0, 0.0, -0.008333333333333)

In [ ]:
# If we are short of memory do we need mean and sd? (if so we'll have to tile: not yet implemented)
SkipMeanAndSD = 1

#### Run the calculations

In [ ]:
res = aggregateContinuous(allData.astype(np.float32), aggFactor, _NDV, 1)

#### Save all as compressed or uncompressed format

In [ ]:
resStructure

In [ ]:
# Uncompressed
for i in range(len(resStructure)):
    stat = resStructure[i]
    savecontResult(thing,stat,'',res[i],_NDV, outDir, True)


In [ ]:
# Or compressed
for i in range(len(resStructure)):
    stat = resStructure[i]
    savecontResult(thing,stat,'_LZW',res[i],_NDV, outDir, False)


In [ ]:
# Or save manually if we've skipped mean / sd
savecontResult('HS360Deg_1k','Count','',res[0],_NDV, outDir, False)
savecontResult('HS360Deg_1k','Max','',res[1],_NDV, outDir, False)
savecontResult('HS360Deg_1k','Min','',res[3],_NDV, outDir, False)
savecontResult('HS360Deg_1k','Range','',res[4],_NDV, outDir, False)
savecontResult('HS360Deg_1k','Sum','',res[5],_NDV, outDir, False)


## Example: run for multiple continuous datasets with different names

In [ ]:
dataPaths = {
'EVI_Mean': r'G:\NewStats\MG_Aligned\EVI_Mean_From_Monthly.tif',
'LST_Day_Mean': r'G:\NewStats\MG_Aligned\LST_Day_Mean_From_Monthly.tif',
'LST_Night_Mean': r'G:\NewStats\MG_Aligned\LST_Night_Mean_From_Monthly.tif',
'TCW_Mean': r'G:\NewStats\MG_Aligned\TCW_Mean_From_Monthly.tif',
'TCB_Mean': r'G:\NewStats\MG_Aligned\TCB_Mean_From_Monthly.tif',

'EVI_SD': r'G:\NewStats\MG_Aligned\EVI_SD_From_Daily.tif',
'LST_Day_SD': r'G:\NewStats\MG_Aligned\LST_Day_SD_From_Daily.tif',
'LST_Night_SD': r'G:\NewStats\MG_Aligned\LST_Night_SD_From_Daily.tif',
'TCW_SD': r'G:\NewStats\MG_Aligned\TCW_SD_From_Daily.tif',
'TCB_SD': r'G:\NewStats\MG_Aligned\TCB_SD_From_Daily.tif',}

In [ ]:
outDir = r'G:\SynopticData\MG_Aligned\5km\tmp'

In [ ]:
for thing in dataPaths:
    ds = gdal.Open(dataPaths[thing])
    bnd = ds.GetRasterBand(1)
    _NDV = bnd.GetNoDataValue()
    inputGT = ds.GetGeoTransform()
    inputProj = ds.GetProjection()
    outputGT = (inputGT[0], inputGT[1]*aggFactor, 0.0, inputGT[3], 0.0, inputGT[5] * aggFactor)
    print ds.GetDescription()
    allData = bnd.ReadAsArray()
    allData[np.isnan(allData)] = _NDV
    res = aggregateContinuous(allData, aggFactor, _NDV)
    for i in range (len(resStructure)):
        stat = resStructure[i]
        savecontResult(thing,stat,'',res[i],_NDV, outDir, None, True)
        savecontResult(thing,stat,'_LZW',res[i],_NDV, outDir, None, False)

In [42]:
dataPaths = glob.glob(r'C:\Temp\dataprep\test_invert_arid.tif')
ds = gdal.Open(dataPaths[0])
b = ds.GetRasterBand(1)

In [43]:
ds.RasterXSize, ds.RasterYSize

(43200, 17400)

In [44]:
b.GetNoDataValue()

-128.0

In [70]:
#self, xSizeIn, ySizeIn, xSizeOut, ySizeOut, _NDV, minMaxRangeSumOnly)
aggomat = RasterAggregator_float(43200,17400,8640,3480,-128,1)

In [71]:
arr1 = b.ReadAsArray(0,0,21600,8700).astype(np.float32)
#(self, float[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
aggomat.addTile(arr1, 0, 0)


In [72]:
arr1 = b.ReadAsArray(21600,0,21600,8700).astype(np.float32)
#(self, float[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
aggomat.addTile(arr1, 21600, 0)

In [73]:
arr1 = b.ReadAsArray(0,8700,21600,8700).astype(np.float32)
#(self, float[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
aggomat.addTile(arr1, 0, 8700)

In [74]:
arr1 = b.ReadAsArray(21600,8700,21600,8700).astype(np.float32)
#(self, float[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
aggomat.addTile(arr1, 21600, 8700)

In [75]:
r = aggomat.GetResults()

In [81]:
r['count'].shape

(3480L, 8640L)

## Example: run for multiple continuous datasets in a series

In [9]:
# for example some output monthly data cubes at 1km
#dataPaths = glob.glob(r'G:\Extra\Output\Aggregated\1km\LST*.tif')
#dataPaths = glob.glob(r'G:\DataPrep\population\GRUMP\tif\GRUMP*.tif')
#dataPaths = glob.glob(r'F:\MOD11A2_Gapfilled_Output\LST_Day\Output_Final_30k_2030pc\*Data.tif')
#dataPaths = glob.glob(r'F:\MOD11A2_Input_Mosaics\Day\*_Day.tif')
dataPaths = glob.glob(r'E:\Temp\pop\ihme_figures\02_processing\06_IHME_Corrected_Grids\by_gender\*.tif')
#dataPaths = glob.glob(r'E:\Temp\pop\ihme_figures\04_outputs\ihme_pop_clipped_2016_limits\*.tif')

In [ ]:
fn = '.'.join(os.path.basename(dataPaths[0]).split(os.path.extsep)[:-1])
fn = fn.replace('.1km.Data', '.5km.Mean')
fn = fn.replace('1km', '5km')
fn = fn + '_5km'
fn

In [11]:
#outDir = r'G:\Extra\Output\Aggregated\5km'
outDir = r'G:\DataPrep\population\GRUMP\tif'
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\hsg\MODIS_Data\5km_8day\LST_Day'
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\hsg\MODIS_Data\5km_8day_Unfilled\LST_Day'
outDir = r'E:\Temp\pop\ihme_figures\04_outputs\ihme_pop_clipped_2016_limits\5k_sum'
outDir = r'E:\Temp\pop\ihme_figures\02_processing\06_IHME_Corrected_Grids\by_gender\5km_sum'

In [ ]:
dataPaths

##### Here we are saving only the mean grid from the aggregation stats, which is all that's kept for the 5km cubes

In [ ]:
if not os.path.exists(outDir):
    os.makedirs(outDir)
for filename1k in dataPaths:
    ds = gdal.Open(filename1k)
    bnd = ds.GetRasterBand(1)
    _NDV = bnd.GetNoDataValue()
    
    inputGT = ds.GetGeoTransform()
    inputProj = ds.GetProjection()
    outputGT = (inputGT[0], inputGT[1]*aggFactor, 0.0, inputGT[3], 0.0, inputGT[5] * aggFactor)
    print ds.GetDescription()
    allData = bnd.ReadAsArray()
    
    dTypeIn = allData.dtype
    if dTypeIn != np.float32:
        allData = allData.astype(np.float32)
        dTypeChanged = True
    else:
        dTypeChanged = False
    
    if _NDV:
        allData[np.isnan(allData)] = _NDV
        res = aggregateContinuous(allData, aggFactor, _NDV)
    else:
        res = aggregateContinuous(allData, aggFactor)
    resMean = res["mean"]
    resMin = res["min"]
    resMax = res["max"]
    if dTypeChanged:
        resMean = resMean.astype(dTypeIn)
        resMin = resMin.astype(dTypeIn)
        resMax = resMax.astype(dTypeIn)
    
    outFNStub = os.path.extsep.join(os.path.basename(filename1k).split(os.path.extsep)[:-1])
    outFNStub = outFNStub.replace('.1km.Data', '.5km.Mean')
    outFNStub = outFNStub.replace('1km', '5km')
    outFNBase = outFNStub + "_5km_Unfilled_Mean"
    #fn = fn + '_5km_Unfilled
    #savecontResult('', '', '', resMean, _NDV, outDir, outFNBase, False)
    outFNBase = outFNStub + "_5km_Max"
    savecontResult('', '', '', resMax, _NDV, outDir, outFNBase, False)
    outFNBase = outFNStub + "_5km_Min"
    savecontResult('', '', '', resMin, _NDV, outDir, outFNBase, False)

In [ ]:
if not os.path.exists(outDir):
    os.makedirs(outDir)
for filename1k in dataPaths:
    ds = gdal.Open(filename1k)
    bnd = ds.GetRasterBand(1)
    _NDV = bnd.GetNoDataValue()
    
    inputGT = ds.GetGeoTransform()
    inputProj = ds.GetProjection()
    outputGT = (inputGT[0], inputGT[1]*aggFactor, 0.0, inputGT[3], 0.0, inputGT[5] * aggFactor)
    print ds.GetDescription()
    allData = bnd.ReadAsArray()
    
    dTypeIn = allData.dtype
    if dTypeIn != np.float32:
        allData = allData.astype(np.float32)
        dTypeChanged = True
    else:
        dTypeChanged = False
    
    if _NDV:
        allData[np.isnan(allData)] = _NDV
        #res = aggregateContinuous(allData, aggFactor, _NDV)
        res = aggregateContinuous(allData, aggFactor, _NDV,  minMaxRangeSumOnly = 1)
    else:
        res = aggregateContinuous(allData, aggFactor, None, minMaxRangeSumOnly = 1)
    resSum = res["sum"]
    if dTypeChanged:
        resSum = resSum.astype(dTypeIn)
        
    outFNStub = os.path.extsep.join(os.path.basename(filename1k).split(os.path.extsep)[:-1])
    outFNStub = outFNStub.replace('.1km.Data', '.5km.Mean')
    outFNStub = outFNStub.replace('1km', '5km')
    outFNBase = outFNStub + "_5km_Sum"
    #fn = fn + '_5km_Unfilled
    savecontResult('', '', '', resSum, _NDV, outDir, outFNBase, False)
    

## Example: run for a single categorical dataset

#### Provide category numbers (raster values) and names

In [11]:
# The values that the input raster has, which should start from zero
ibgpCats = {
    0:'Water',
    1:'Evergreen_Needleleaf_Forest',
    2:'Evergreen_Broadleaf_Forest',
    3:'Deciduous_Needleleaf_Forest',
    4:'Deciduous_Broadleaf_Forest',
    5:'Mixed_Forest',
    6:'Closed_Shrublands',
    7:'Open_Shrublands',
    8:'Woody_Savannas',
    9:'Savannas',
    10:'Grasslands',
    11:'Permanent_Wetlands',
    12:'Croplands',
    13:'Urban_And_Built_Up',
    14:'Cropland_Natural_Vegetation_Mosaic',
    15:'Snow_And_Ice',
    16:'Barren_Or_Sparsely_Populated',
    17:'Unclassified', # =254 in source data, hardcoded a remap in the cython
    18:'No_Data' # =255 in source data
}

In [12]:
sorted(ibgpCats)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [13]:
nCats = len(ibgpCats)

#### Run the calculations

In [ ]:
#%%timeit
# global calculation takes approx 16 seconds
res = aggregateStats(allData,10,len(ibgpCats))

#### Write categorical results

In [ ]:
# write the results to individual (one per class) tiff files
outDrv = gdal.GetDriverByName('GTiff')
outNameTemplate = r'G:\MCD12Q1\Proportional\5km\{0!s}_{1!s}_5km_{2!s}.tif'
for i in range(0,len(ibgpCats)):
    className = ibgpCats[i]
    outRasterName = outNameTemplate.format(i,className,"Percentage")
    outRaster = outDrv.Create(outRasterName,res[0].shape[2], res[0].shape[1],1,gdal.GDT_Byte,
                              ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
    outRaster.SetGeoTransform(outputGT)
    outRaster.SetProjection(inputProj)
    outBand = outRaster.GetRasterBand(1)
    outBand.WriteArray(res[0][i])
    outBand.FlushCache()
    del outBand
    outRasterName = outNameTemplate.format(i,className,"LikeAdjacencies")
    outRaster = outDrv.Create(outRasterName,res[1].shape[2], res[1].shape[1],1,gdal.GDT_Float32,
                              ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
    outRaster.SetGeoTransform(outputGT)
    outRaster.SetProjection(inputProj)
    outBand = outRaster.GetRasterBand(1)
    outBand.WriteArray(res[1][i])
    outBand.FlushCache()
    del outBand
    outRaster = None
outRasterName = outNameTemplate.format("Overall", "Class", "Majority")
outRaster = outDrv.Create(outRasterName,res[2].shape[1], res[2].shape[0],
                          1,gdal.GDT_Byte,
                          ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES",
                           "COMPRESS=LZW","PREDICTOR=2"])
outRaster.SetGeoTransform(outputGT)
outRaster.SetProjection(inputProj)
outBand = outRaster.GetRasterBand(1)
outBand.WriteArray(res[2])
outBand.FlushCache()
del outBand
outRaster = None
# To enforce alignment with the mastergrids run the following on each file:
# gdal_edit %filename.tif% -a_ullr -180 89.99994 179.9998560 -89.999988

## Example: run for multiple categorical datasets in a series

In [5]:
dataPaths = glob.glob(r'G:\MCD12Q1\500m\A2013*.tif')

In [6]:
fn = os.path.basename(dataPaths[0]).split(os.path.extsep)[0]
fn = fn.replace('_1km_Data', '.5km.Mean')
fn = fn.replace('_1km_FilledProportion', '.5km.FilledProportion')
fn = fn.replace('Night_', 'Night.')
fn = fn.replace('-', '.')
fn

'A2013001_MCD12Q1'

In [7]:
outDir = r'G:\MCD12Q1\5km'

In [8]:
aggFactor = 10

In [9]:
ds = gdal.Open(dataPaths[0])
bnd = ds.GetRasterBand(1)
_NDV = bnd.GetNoDataValue()
inputGT = ds.GetGeoTransform()
inputProj = ds.GetProjection()
ds = None
outputGT = (inputGT[0], inputGT[1]*aggFactor, 0.0, inputGT[3], 0.0, inputGT[5] * aggFactor)

In [14]:
# write the results to individual (one per class) tiff files
for filename500m in dataPaths:
    inFN = os.path.basename(filename500m).split(os.path.extsep)[0]
    yr = inFN[1:5]
    outDrv = gdal.GetDriverByName('GTiff')
    
    outNameTemplate = r'{0!s}\{1!s}\{2!s}.{3!s}_{4!s}.5km.{5!s}.tif'
    
    likeAdjNDV = -9999
    with rasterio.open(filename500m) as src:
        allData = src.read_band(1, masked=False)
    res = aggregateStats(allData,aggFactor,len(ibgpCats), likeAdjNDV)
    
    for i in range(0,len(ibgpCats)):
        className = ibgpCats[i]
        outRasterName = outNameTemplate.format(
            outDir, 
            "Proportional", 
            yr,
            "Class"+str(i).zfill(2), 
            className, 
            "Percentage"
        )
        outRaster = outDrv.Create(outRasterName,res[0].shape[2], res[0].shape[1],1,gdal.GDT_Byte,
                                  ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
        outRaster.SetGeoTransform(outputGT)
        outRaster.SetProjection(inputProj)
        outBand = outRaster.GetRasterBand(1)
        outBand.WriteArray(res[0][i])
        outBand.FlushCache()
        del outBand
        
        #outRasterName = outNameTemplate.format(outDir, i, className, "LikeAdjacencies")
        outRasterName = outNameTemplate.format(
            outDir, 
            "LikeAdjacencies", 
            yr,
            "Class"+str(i).zfill(2), 
            className, 
            "LikeAdjacencies"
        )
        outRaster = outDrv.Create(outRasterName,res[1].shape[2], res[1].shape[1],1,gdal.GDT_Float32,
                                  ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES","COMPRESS=LZW","PREDICTOR=2"])
        outRaster.SetGeoTransform(outputGT)
        outRaster.SetProjection(inputProj)
        outBand = outRaster.GetRasterBand(1)
        outBand.SetNoDataValue(likeAdjNDV)
        outBand.WriteArray(res[1][i])
        outBand.FlushCache()
        del outBand
        outRaster = None
        
    #outRasterName = outNameTemplate.format("Overall", "Class", "Majority")
    outRasterName = outNameTemplate.format(
            outDir, 
            "Majority", 
            yr,
            "Overall", 
            "Class", 
            "Majority"
        )
    outRaster = outDrv.Create(outRasterName,res[2].shape[1], res[2].shape[0],
                              1,gdal.GDT_Byte,
                              ["TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES",
                               "COMPRESS=LZW","PREDICTOR=2"])
    outRaster.SetGeoTransform(outputGT)
    outRaster.SetProjection(inputProj)
    outBand = outRaster.GetRasterBand(1)
    outBand.WriteArray(res[2])
    outBand.FlushCache()
    del outBand
    outRaster = None
    # To enforce alignment with the mastergrids run the following on each file:
    # gdal_edit %filename.tif% -a_ullr -180 89.99994 179.9998560 -89.999988

In [18]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
import numpy as np
cimport openmp
from cython.parallel import parallel, prange

cdef class RasterAggregator_Categorical:
    ''' Aggregates a categorical raster grid by a specified factor (e.g. 10 to aggegate 500m to 5km data)

     Returns a tuple containing two three dimensional grids and one two dimensional grid
     at the aggregated resolution, each representing a different summary of the source
     pixels covered by each output pixel, namely:
        (
          Fraction (3D),
          Like-Adjacency (3D),
          Majority (2D)
        )
     The fraction and like adjacency outputs have one image (in the z dimension) for each category
     (value) of the input raster. It's assumed that the input has values from zero and the mapping
     is therefore between the input value and the output position, i.e. an input pixel of value 4
     will contribute to the 4th grid in the output stack. This is suitable for use with IGBP
     landcover data.

     Note that this means the output will have as many "layers" in the z dimension as there are unique
     values in the input data. Thus this is only recommended where there are a small number of unique
     values, or the output grids are small, due to the large memory implications.

     The input grid must (in this implementation) have dimensions that are
     exact multiples of aggregation factor. The input grid must be of unsigned integer type.
    '''

    cdef:
       Py_ssize_t xShapeIn, yShapeIn, tileXShapeIn, tileYShapeIn
       Py_ssize_t xShapeOut, yShapeOut
       double xFact, yFact
       float proportion

    cdef:
        Py_ssize_t xIn, yIn, xOut, yOut, catNum
        int yBelow, yAbove, xLeft, xRight


    cdef:
        unsigned char nCategories, _NDV
        float[:,:,::1] outputFracArr
        float[:,:,::1] outputLikeAdjArr
        unsigned char [:,::1] outputMajorityArr

    cdef:
        float [:,::1] tmpMajorityPropArr
        char[:,::1] _coverageArr

    def __cinit__(self, xSizeIn, ySizeIn, xSizeOut, ySizeOut, unsigned char nCategories, _NDV):
        assert xSizeIn > xSizeOut
        assert ySizeIn > ySizeOut

        self.xShapeIn = xSizeIn
        self.yShapeIn = ySizeIn
        self.xShapeOut = xSizeOut
        self.yShapeOut = ySizeOut

        self.xFact = <double>self.xShapeIn / self.xShapeOut
        self.yFact = <double>self.yShapeIn / self.yShapeOut

        self._NDV = _NDV
        self.nCategories = nCategories

        self.outputFracArr = np.zeros(shape = (nCategories, self.yShapeOut, self.xShapeOut),
                                      dtype = np.float32)
        self.outputLikeAdjArr = np.zeros(shape = (nCategories, self.yShapeOut, self.xShapeOut),
                                         dtype = np.float32)
        self.outputMajorityArr = np.zeros(shape = (self.yShapeOut, self.xShapeOut),
                                          dtype = np.uint8)
        self.tmpMajorityPropArr = np.zeros(shape = (self.yShapeOut, self.xShapeOut),
                                           dtype = np.float32)
        self._coverageArr = np.zeros(shape = (self.yShapeOut, self.xShapeOut), dtype = np.byte)

    cpdef addTile(self, unsigned char[:,::1] data, Py_ssize_t xOffset, Py_ssize_t yOffset):
        cdef:
            #shape of the data we receive
            Py_ssize_t tileYShapeIn, tileXShapeIn
            # coords of neighbours in global grid
            int yBelowGlobal, yAboveGlobal, xLeftGlobal, xRightGlobal
            # coords of neighbours in the received tile
            int yBelowTile, yAboveTile, xLeftTile, xRightTile
            # current pixel coords in global and tile coords
            Py_ssize_t xInGlobal, xInTile, yInGlobal, yInTile
            unsigned char localValue
            unsigned char nNeighbours
            unsigned char catNum
            float likeAdjProp
            Py_ssize_t xOut, yOut
            # how much of an output cell does each input cell account for
            float proportion
            char isOk = 1

        tileXShapeIn = data.shape[1]
        tileYShapeIn = data.shape[0]

        # how much of an output cell does each input cell account for
        proportion = 1.0 / (self.xFact * self.yFact)


        with nogil, parallel():
            for yInTile in prange (tileYShapeIn):
                yInGlobal = yInTile + yOffset

                yAboveGlobal = yInGlobal - 1
                if yInGlobal == 0:
                    yAboveGlobal = -1
                yAboveTile = yInTile - 1
                if yInTile == 0:
                    yAboveTile = -1

                yBelowGlobal = yInGlobal + 1
                if yInGlobal == self.yShapeIn - 1:
                    yBelowGlobal = -1
                yBelowTile = yInTile + 1
                if yInTile == tileYShapeIn - 1:
                    yBelowTile = -1

                yOut = <int> (yInGlobal / self.yFact)
                # yeah i know that it's an unsigned char but we won't actually use this value,
                # these assignments are to cause the cython converter to make these thread-local
                localValue = -1
                xOut = -1

                for xInTile in range(tileXShapeIn):
                    xInGlobal = xInTile + xOffset
                    nNeighbours = 0
                    likeAdjProp = 0

                    xLeftGlobal = xInGlobal - 1
                    if xInGlobal == 0:
                        xLeftGlobal = -1
                    xLeftTile = xInTile - 1
                    if xInTile == 0:
                        xLeftTile = -1

                    xRightGlobal = xInGlobal + 1
                    if xInGlobal == self.xShapeIn - 1:
                        xRightGlobal = -1
                    xRightTile = xInTile + 1
                    if xInTile == tileXShapeIn - 1:
                        xRightTile = -1

                    xOut = <int> (xInGlobal / self.xFact)

                    self._coverageArr[yOut, xOut] = 1
                    localValue = data[yInTile, xInTile]
                    if localValue == self._NDV:
                        # don't do anything
                        continue
                    if localValue > self.nCategories + 1:
                        isOk = 0
                        break

                    # the fraction is straightforward, just the proportion of the output cell
                    # covered by this one
                    self.outputFracArr[localValue, yOut, xOut] += proportion

                    # the like adjacency contribution of a given incoming cell
                    # is less straightforward because it depends on neighbours and at
                    # edges of a tile we don't have access to all neighbours of incoming data
                    # so we just calculate it based on how many neighbours we do have
                    # (this isn't totally accurate of course)
                    if yAboveTile >= 0:
                        nNeighbours += 1
                        if data[yAboveTile, xInTile] == localValue:
                            likeAdjProp += 1
                    if xRightTile >= 0:
                        nNeighbours += 1
                        if data[yInTile, xRightTile] == localValue:
                            likeAdjProp += 1
                    if yBelowTile >= 0:
                        nNeighbours += 1
                        if data[yBelowTile, xInTile] == localValue:
                            likeAdjProp += 1
                    if xLeftTile >= 0:
                        nNeighbours += 1
                        if data[yInTile, xLeftTile] == localValue:
                            likeAdjProp += 1
                    #likeAdjProp = likeAdjProp / nNeighbours
                    self.outputLikeAdjArr[localValue, yOut, xOut] += (
                        (likeAdjProp / nNeighbours) * proportion)
        if not isOk:
            print "A value was encountered that was greater than the number of categories expected"
        

    cdef finalise(self):
        cdef:
            Py_ssize_t xOut, yOut
            float iscomplete = 1
        for catNum in range (self.nCategories):
            yOut = -1
            xOut = -1
            for yOut in range (self.yShapeOut):
                xOut = -1
                for xOut in range (self.xShapeOut):
                    if self._coverageArrArr[yOut, xOut] == 0:
                        iscomplete = 0
                    if self.outputFracArr[catNum, yOut, xOut] > 0:
                        self.outputLikeAdjArr[catNum, yOut, xOut] = (
                            self.outputLikeAdjArr[catNum, yOut, xOut] / self.outputFracArr[catNum, yOut, xOut]
                        )
                    else:
                        self.outputLikeAdjArr[catNum, yOut, xOut] = self._NDV

                    if self.outputFracArr[catNum, yOut, xOut] > self.tmpMajorityPropArr[yOut, xOut]:
                        self.tmpMajorityPropArr[yOut, xOut] = self.outputFracArr[catNum, yOut, xOut]
                        self.outputMajorityArr[yOut, xOut] = catNum

                    self.outputFracArr[catNum, yOut, xOut] *= 100
                    self.outputFracArr[catNum, yOut, xOut] += 0.5

        if not iscomplete:
            print "Warning, generating a result without having received input data for full extent"
            return False
        return True

    cpdef GetResults(self):
        '''If input is complete, returns an object containing 'fractions', 'likeadjacencies', 'majority' '''
        if not self.finalise():
            return None
        return {
            "fractions": np.asarray(self.outputFracArr).astype(np.uint8),
            "likeadjacencies": np.asarray(self.outputLikeAdjArr),
            "majority": np.asarray(self.outputMajorityArr) #.astype(np.uint8)
        }















In [ ]:
def catRunner(dataPaths, categories):
    '''Run the aggregation code for each file specifed in dataPaths which should be a list of tiff files.
    
    The global notebook variables outDir, method, minMaxRangeSumOnly, itemsToSave, and 
    (aggregationFactor OR resolution OR requiredXSize and requiredYSize)
    should be set first, along with the fnGetter function for producing output filenames.
    '''
    for f in dataPaths:
        print f
        ds = gdal.Open(f)
        b = ds.GetRasterBand(1)
        ndv = b.GetNoDataValue()
        if ndv is None:
            print "no ndv"
            ndv = -9999
        inputGT = ds.GetGeoTransform()
        inputProj = ds.GetProjection()
        
        assert len(categories) <= 256
        assert max(categories) < 256
        assert min(categories) >= 0
        
        nCategories = len(categories)
        
        nBytesRequired = ds.RasterXSize * ds.RasterYSize * 1 * nCategories

        outGT, outShape = calcAggregatedProperties(method, (ds.RasterYSize, ds.RasterXSize), 
                                                   inputGT, aggregationFactor, 
                                                   (requiredYSize, requiredXSize), 
                                                   resolution)
        outYSize, outXSize = outShape    
        tiles = getTiles(ds.RasterXSize, ds.RasterYSize, 20000)
        
        aggregator = RasterAggregator_Categorical(ds.RasterXSize, ds.RasterYSize, 
                                            outXSize, outYSize,float(ndv), 
                                            minMaxRangeSumOnly)
        print "Running {0!s} tiles".format(len(tiles)),
        for tile in tiles:
            print ".",
            xoff = tile[0][0]
            yoff = tile[1][0]
            xsize = tile[0][1] - xoff
            ysize = tile[1][1] - yoff
            inArr = b.ReadAsArray(xoff, yoff, xsize, ysize).astype(np.uint8)
            aggregator.addTile(inArr, xoff, yoff)
        r = aggregator.GetResults()
        itemstosave = ['fractions','likeadjacencies','majority']
        for i in itemstosave:
            fnOut = fnGetter(os.path.basename(f), i)
            print fnOut
            # the file-saving function will save to a tiff of datatype matching the array
            # it receives.
            if i in ['fractions','max','range']:
                # if the input was some integer type then save as this, even though the 
                # aggregation code always outputs float32
                nptype = gdal_array.GDALTypeCodeToNumericTypeCode(b.DataType)
                savecontResult(r[i].astype(nptype), ndv, outGT, inputProj, outDir, fnOut)
            elif i in ['mean','sd', 'sum']:
                # sum might be integer but potentially of larger type than the input, don't bother
                # dealing with conversion for now
                savecontResult(r[i], ndv, outGT, inputProj, outDir, fnOut)
            elif i in ['count']:
                savecontResult(r[i].astype(np.int32), ndv, outGT, inputProj, outDir, fnOut)
            else:
                assert False
                
        

        